<h1>this assigment is comprised of the parts named as first,second and third assigments.</h1>

<h1>First part of assignment</h1>

In [1]:
import pandas as pd
import numpy as np
import types
import pandas as pd
import requests
from bs4 import BeautifulSoup
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f50bbb71b92d4d9c8e4292ff7e2d6cc2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='l78LODgCJNrkRURB0PjL4iIiLyumg6Q0JjgbUZK8bVTp',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_f50bbb71b92d4d9c8e4292ff7e2d6cc2.get_object(Bucket='webscrapingproject-donotdelete-pr-5dijb8earp6i9x',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [2]:
pip install branca

Note: you may need to restart the kernel to use updated packages.


<strong>BeautifulSoup library is used for getting table data in html and I removed the "Not assigned" in Borough columns if exists</strong>

In [3]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.find_all('table')
df = pd.read_html(str(table))[0]

df = df[df.Borough != 'Not assigned']

In [4]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
print(df.shape)

(103, 3)


<h1>Second part of assignment</h1>

<strong>the csv files is used for the data of latitude and longitude</strong>

In [6]:
dfGeo=df_data_1.copy()
dfGeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<strong>the values of longitude and latitude are merged with the table neighbourhood</strong>

In [7]:
#listOfGeo=dfGeo["Postal Code"].to_list()
df["Latitude"]=0
df["Longitude"]=0
for item in dfGeo["Postal Code"]:
    data=dfGeo[dfGeo["Postal Code"]==item]
    df.loc[df['Postal Code'] == item,['Latitude']]=data["Latitude"].values[0]
    df.loc[df['Postal Code'] == item,['Longitude']]=data["Longitude"].values[0]
    

  

In [8]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h1>Part 3 of the assignment</h1>

In [9]:
from sklearn.cluster import KMeans 
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import folium
%matplotlib inline 


In [10]:
from geopy.geocoders import Nominatim

<strong>obtain the latitude and longitude values via geocode</strong>

In [11]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h3>show the neighbourhoods on the map</h3>

In [12]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<strong>the Borough values which contain "Toronto" are used</strong>

In [13]:
torontoDataframe=df[df["Borough"].str.contains("Toronto")].reset_index(drop=True)
torontoDataframe.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


<strong>the info for using foursquare API</strong>

In [14]:
CLIENT_ID = '1ZRVG3SNGE24M51YROR4BAZQCNDBGHBOLSMLV4MJ2SRNTJMO' # your Foursquare ID
CLIENT_SECRET = 'P1I5450KIXFMQRLP1V0YJDBF3QXPFWXVI3RO12E3ZW2DYZME' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=50
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1ZRVG3SNGE24M51YROR4BAZQCNDBGHBOLSMLV4MJ2SRNTJMO
CLIENT_SECRET:P1I5450KIXFMQRLP1V0YJDBF3QXPFWXVI3RO12E3ZW2DYZME


In [15]:
torontoDataframe.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

<strong>the function for getting cayegory type of venue</strong>

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<strong>the function for getting the values of the nearby venues</strong>

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3>we get the json file to get the properties of neighbourhood such as venue venu category</h3>

In [ ]:
toronto_venues = getNearbyVenues(names=torontoDataframe['Neighbourhood'],
                                   latitudes=torontoDataframe['Latitude'],
                                   longitudes=torontoDataframe['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West


In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

<strong> printing out the venues and venue category by grouping the items in neighbourhood</strong>

In [ ]:
venueCount=toronto_venues.groupby('Neighbourhood').count()
venueCount[["Venue","Venue Category"]].head()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

<h3> getting the cluster values that I want to to use in the kmeans algorithm</h3>

In [ ]:
print(toronto_venues['Venue Category'].unique()[0:5])

tobeClustered=["Coffee Shop","Pub","Mediterranean Restaurant","Shopping Mall"]

<strong>For the classification, the dataframe converted into a table containing the data 1 and 0 to show whether the venue category exist in the neighbourhood</strong>

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_to_be_clustred=toronto_onehot[["Neighbourhood","Coffee Shop","Pub","Mediterranean Restaurant","Shopping Mall"]]
print(toronto_to_be_clustred.shape)
toronto_to_be_clustred.head(3)


In [ ]:
toronto_to_be_clustred.shape

<h3>By getting the means, we can check the similarities among the neighbourhoods</h3>

In [ ]:
toronto_grouped = toronto_to_be_clustred.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
toronto_grouped.shape

<h3>the first grapph the shows the accuarcy based on the value of k </h3>

In [ ]:
distortions = []
toronto_grouped_clustering_for_bestK = toronto_grouped.drop('Neighbourhood', 1)
K = range(1,20)
for k in K:
    kmean = KMeans(n_clusters=k, random_state=0, n_init = 50, max_iter = 1000)
    kmean.fit(toronto_grouped_clustering_for_bestK)
    distortions.append(kmean.inertia_)


plt.figure(figsize=(10,5))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method')
plt.show()


<strong> I used a KElbowVisualizer function to find the best K value for kmeans clustering
The function link as :https://www.scikit-yb.org/en/latest/api/cluster/elbow.html
</strong>

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

from yellowbrick.cluster import KElbowVisualizer

# Generate synthetic dataset with 8 random clusters
#X, y = make_blobs(n_samples=1000, n_features=12, centers=8, random_state=42)

# Instantiate the clustering model and visualizer
kmean = KMeans(n_clusters=k, random_state=42, n_init = 50, max_iter = 1000)
#kmean.fit(toronto_grouped_clustering_for_bestK)


visualizer = KElbowVisualizer(kmean, k=(2,12))

visualizer.fit(toronto_grouped_clustering_for_bestK)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure


In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:

toronto_grouped.insert(0, 'Cluster_Labels', kmeans.labels_)
toronto_merged = torontoDataframe

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged1 = toronto_merged.join(toronto_grouped.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged1.head() # check the last columns!

In [ ]:
toronto_merged1["Cluster_Labels"].value_counts()

<h3>showing the neighbourhoods basod the kmeans clustering</h3>

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#theZeorLabel=toronto_merged1.loc[toronto_merged1['Cluster_Labels'] == 0, toronto_merged1.columns[[1] + list(range(1, toronto_merged1.shape[1]))]]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged1['Latitude'], toronto_merged1['Longitude'], toronto_merged1['Neighbourhood'], toronto_merged1['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Analyzing the cluster labeled as 0</h3>

In [ ]:
toronto_merged1.loc[toronto_merged1['Cluster_Labels'] == 0, toronto_merged1.columns[[1] + list(range(1, toronto_merged1.shape[1]))]]